# <span><img src="./MavenomicsLogoNew.png" alt="Mavenomics Logo" style="margin-bottom:-15px" />&nbsp;|&nbsp; MBTA Realtime Statistics</span>

Boston's transit system, known as the MBTA, offers a [JSON API](https://www.mbta.com/developers) for buses, trains, streetcars, subways, and more. This feed is realtime, and offers positions, next stop predictions, on-time performance, and more. This dashboard allows us to see the overall performance of the system right now, with data usually being updated every 2 minutes.

Requirements:
 - [An API v3 key from the MBTA](https://api-v3.mbta.com)
 - MavenWorks
 - pandas >= 0.23

In [1]:
# Put your API key here!

MBTA_API_KEY = "76469301e6e5411db089cf6a28bebff5"
UPDATE_INTERVAL = 120
API_BASE = "https://api-v3.mbta.com"
# We only list these lines to limit data consumption
LINES = [
    "Red",
    "Orange",
    "Blue",
    "Silver",
    # The 4 branches of the Green line are represented as independent "lines"
    "Green-B",
    "Green-C",
    "Green-D",
    "Green-E"
]

In [2]:
import requests
import pandas as pd
import mavenworks
import matplotlib.pyplot as plt

In [3]:
def make_table(api_result, include_cols=None):
    """Make a dataframe out of a JSON-API query result"""
    if include_cols is None:
        include_cols = []
    records = api_result["data"]
    includes = {}
    for record in api_result["included"]:
        includes[record["id"]] = record["attributes"]
    def make_record_for_row(row):
        row_attrs = {}
        row_attrs.update(row["attributes"])
        relations = row["relationships"]
        for col in include_cols:
            if col not in relations or relations[col]["data"] is None:
                continue
            rel_id = relations[col]["data"]["id"]
            if rel_id not in includes:
                continue
            include_attrs = {col + "_" + key: value for key, value in includes[rel_id].items()}
            row_attrs.update(include_attrs)
            row_attrs[col + "_id"] = rel_id
        return row_attrs
    df = pd.DataFrame.from_records(map(make_record_for_row, records))
    return df

In [4]:
req = requests.get(f"{API_BASE}/vehicles?include=trip&filter[route]={','.join(LINES)}&api_key={MBTA_API_KEY}")
vehicles = make_table(req.json(), ["trip"])

In [5]:
req = requests.get(f"{API_BASE}/predictions?include=schedule,trip,route&filter[route]={','.join(LINES)}&api_key={MBTA_API_KEY}")
predictions = make_table(req.json(), ["schedule", "trip", "route"])

In [6]:
predictions["delay_seconds"] = pd.to_datetime(predictions["departure_time"]) - pd.to_datetime(predictions["schedule_departure_time"])
predictions["delay_seconds"] = predictions["delay_seconds"].dt.total_seconds()
predictions["Direction"] = predictions.apply(lambda row: row["route_direction_names"][row["trip_direction_id"]], axis=1)
vhc_and_pred = pd.merge(vehicles, predictions, on="trip_id")

In [7]:
def get_updated_data(rand):
    global vhc_and_pred
    req = requests.get(f"{API_BASE}/vehicles?include=trip&filter[route]={','.join(LINES)}&api_key={MBTA_API_KEY}")
    vehicles = make_table(req.json(), ["trip"])
    req = requests.get(f"{API_BASE}/predictions?include=schedule,trip,route&filter[route]={','.join(LINES)}&api_key={MBTA_API_KEY}")
    predictions = make_table(req.json(), ["schedule", "trip", "route"])
    predictions["delay_seconds"] = pd.to_datetime(predictions["departure_time"]) - pd.to_datetime(predictions["schedule_departure_time"])
    predictions["delay_seconds"] = predictions["delay_seconds"].dt.total_seconds()
    predictions["Direction"] = predictions.apply(lambda row: row["route_direction_names"][row["trip_direction_id"]], axis=1)
    vhc_and_pred = pd.merge(vehicles, predictions, on="trip_id")
    return vhc_and_pred

In [8]:
# Auto-generated code, do not edit!
_json = __import__("json")
display(_json.loads("{\"application/vnd.maven.layout+json\": {\"layout\":{\"properties\":{\"flexSize\":1,\"horizontal\":false},\"typeName\":0,\"uuid\":\"76c23bcd-c93b-4dcc-b1bb-56cc1bcbcb5d\",\"attachedProperties\":[{\"Fixed Size (px)\":100,\"Stretch\":null},{\"Fixed Size (px)\":null,\"Stretch\":1.6169762137504073}],\"children\":[{\"properties\":{\"horizontal\":true,\"prunable\":true},\"typeName\":0,\"uuid\":\"fe2a90a5-8b7e-405e-8511-b6cd2b572adf\",\"attachedProperties\":[{\"Fixed Size (px)\":630,\"Stretch\":null},{\"Fixed Size (px)\":null,\"Stretch\":0.5669934640522876}],\"children\":[{\"properties\":{\"caption\":\"MavenTitlePart\",\"flexSize\":1,\"showRegion\":true,\"showTitle\":false},\"typeName\":1,\"uuid\":\"f89ff16b-bfd9-4c5a-b719-1c29d0814451\",\"guid\":\"fc4bec04-9659-455c-b7f2-c5b217b22ed4\"},{\"properties\":{\"caption\":\"CheckboxPart\",\"showRegion\":true,\"showTitle\":false},\"typeName\":1,\"uuid\":\"b3f49fbc-baec-4213-93ac-c86033bcdb0b\",\"guid\":\"4482072c-7d80-4e38-9d37-52c525ee70fa\"}]},{\"properties\":{\"flexSize\":1,\"prunable\":true},\"typeName\":0,\"uuid\":\"4c248aaa-4e53-4ce0-b62d-d06a25aa2b7c\",\"attachedProperties\":[{\"Fixed Size (px)\":null,\"Stretch\":1.469387755102041},{\"Fixed Size (px)\":null,\"Stretch\":0.5306122448979592}],\"children\":[{\"properties\":{\"horizontal\":true,\"prunable\":true},\"typeName\":0,\"uuid\":\"dee9bef9-e466-4704-ba9d-6adc3b2155ae\",\"attachedProperties\":[{\"Fixed Size (px)\":null,\"Stretch\":1},{\"Fixed Size (px)\":null,\"Stretch\":1}],\"children\":[{\"properties\":{\"caption\":\"Delay (seconds) By Line and Direction\",\"showTitle\":true},\"typeName\":1,\"uuid\":\"8a1e7d38-a82c-48a7-9c9e-a6c42f8eb379\",\"guid\":\"2dc1fe45-1933-4a46-9b3c-e31f0ee77d10\"},{\"properties\":{\"caption\":\"PivotPart\",\"showRegion\":true},\"typeName\":1,\"uuid\":\"b795bbc5-c21a-4b76-a77c-66d00ea53066\",\"guid\":\"03e008a8-676b-43df-8f45-5005906a59d9\"}]},{\"properties\":{\"caption\":\"AnimationSliderPart\"},\"typeName\":1,\"uuid\":\"116a78d7-686a-4c50-93bd-6d9ab3b4f0e6\",\"guid\":\"b85c483d-96a8-4295-9370-ae1d34cf72f7\"}]}]},\"parts\":{\"03e008a8-676b-43df-8f45-5005906a59d9\":{\"application/vnd.maven.part+json\":{\"name\":\"PivotPart\",\"id\":\"03e008a8-676b-43df-8f45-5005906a59d9\",\"options\":{\"Input Table\":{\"type\":\"JavaScript\",\"expr\":\"/* @rand */\\nawait new Promise(res => setTimeout(res, 5000));\\n\\nreturn mql`SELECT\\n    route_long_name,\\n    longitude,\\n    latitude\\nFROM KernelEval('vhc_and_pred')`;\",\"globals\":[\"rand\"]},\"Config\":{\"typeName\":\"String\",\"value\":\"{\\\"class\\\":\\\"p-Widget\\\",\\\"plugin\\\":\\\"d3_xy_scatter\\\",\\\"row-pivots\\\":\\\"[]\\\",\\\"column-pivots\\\":\\\"[\\\\\\\"route_long_name\\\\\\\"]\\\",\\\"filters\\\":\\\"[]\\\",\\\"sort\\\":\\\"[]\\\",\\\"style\\\":\\\"position: absolute; z-index: 0; top: 0px; left: 0px; width: 100%; height: 100%;\\\",\\\"view\\\":\\\"d3_xy_scatter\\\",\\\"columns\\\":\\\"[\\\\\\\"latitude\\\\\\\",\\\\\\\"longitude\\\\\\\"]\\\",\\\"aggregates\\\":\\\"{\\\\\\\"route_long_name\\\\\\\":\\\\\\\"count\\\\\\\",\\\\\\\"latitude\\\\\\\":\\\\\\\"sum\\\\\\\",\\\\\\\"longitude\\\\\\\":\\\\\\\"sum\\\\\\\"}\\\",\\\"render_time\\\":\\\"40.13499990105629\\\",\\\"settings\\\":\\\"true\\\"}\"}}},\"text/plain\":\"VisualEditorPart\"},\"2dc1fe45-1933-4a46-9b3c-e31f0ee77d10\":{\"application/vnd.maven.part+json\":{\"name\":\"PivotPart\",\"id\":\"2dc1fe45-1933-4a46-9b3c-e31f0ee77d10\",\"options\":{\"Input Table\":{\"type\":\"Eval\",\"expr\":\"get_updated_data(@rand)\",\"globals\":[\"rand\"]},\"Config\":{\"typeName\":\"String\",\"value\":\"{\\\"class\\\":\\\"p-Widget\\\",\\\"plugin\\\":\\\"d3_heatmap\\\",\\\"row-pivots\\\":\\\"[\\\\\\\"route_long_name\\\\\\\"]\\\",\\\"column-pivots\\\":\\\"[\\\\\\\"Direction\\\\\\\"]\\\",\\\"filters\\\":\\\"[]\\\",\\\"sort\\\":\\\"[]\\\",\\\"style\\\":\\\"position: absolute; z-index: 0; top: 0px; left: 0px; width: 100%; height: 100%;\\\",\\\"view\\\":\\\"d3_heatmap\\\",\\\"columns\\\":\\\"[\\\\\\\"delay_seconds\\\\\\\"]\\\",\\\"aggregates\\\":\\\"{\\\\\\\"current_status\\\\\\\":\\\\\\\"distinct count\\\\\\\",\\\\\\\"delay_seconds\\\\\\\":\\\\\\\"avg\\\\\\\",\\\\\\\"label\\\\\\\":\\\\\\\"distinct count\\\\\\\",\\\\\\\"route_color\\\\\\\":\\\\\\\"distinct count\\\\\\\",\\\\\\\"route_description\\\\\\\":\\\\\\\"distinct count\\\\\\\",\\\\\\\"route_direction_destinations\\\\\\\":\\\\\\\"distinct count\\\\\\\",\\\\\\\"route_direction_names\\\\\\\":\\\\\\\"distinct count\\\\\\\",\\\\\\\"route_fare_class\\\\\\\":\\\\\\\"distinct count\\\\\\\",\\\\\\\"route_id\\\\\\\":\\\\\\\"distinct count\\\\\\\",\\\\\\\"route_long_name\\\\\\\":\\\\\\\"distinct count\\\\\\\",\\\\\\\"route_short_name\\\\\\\":\\\\\\\"distinct count\\\\\\\",\\\\\\\"route_text_color\\\\\\\":\\\\\\\"distinct count\\\\\\\",\\\\\\\"schedule_arrival_time\\\\\\\":\\\\\\\"distinct count\\\\\\\",\\\\\\\"schedule_departure_time\\\\\\\":\\\\\\\"distinct count\\\\\\\",\\\\\\\"schedule_direction_id\\\\\\\":\\\\\\\"distinct count\\\\\\\",\\\\\\\"schedule_drop_off_type\\\\\\\":\\\\\\\"distinct count\\\\\\\",\\\\\\\"schedule_id\\\\\\\":\\\\\\\"distinct count\\\\\\\",\\\\\\\"schedule_pickup_type\\\\\\\":\\\\\\\"distinct count\\\\\\\",\\\\\\\"schedule_relationship\\\\\\\":\\\\\\\"distinct count\\\\\\\",\\\\\\\"schedule_stop_sequence\\\\\\\":\\\\\\\"distinct count\\\\\\\",\\\\\\\"schedule_timepoint\\\\\\\":\\\\\\\"distinct count\\\\\\\",\\\\\\\"speed\\\\\\\":\\\\\\\"count\\\\\\\",\\\\\\\"status\\\\\\\":\\\\\\\"distinct count\\\\\\\",\\\\\\\"trip_block_id_x\\\\\\\":\\\\\\\"distinct count\\\\\\\",\\\\\\\"trip_block_id_y\\\\\\\":\\\\\\\"distinct count\\\\\\\",\\\\\\\"trip_headsign_x\\\\\\\":\\\\\\\"dominant\\\\\\\",\\\\\\\"trip_headsign_y\\\\\\\":\\\\\\\"distinct count\\\\\\\",\\\\\\\"trip_id\\\\\\\":\\\\\\\"distinct count\\\\\\\",\\\\\\\"trip_name_x\\\\\\\":\\\\\\\"distinct count\\\\\\\",\\\\\\\"trip_name_y\\\\\\\":\\\\\\\"distinct count\\\\\\\",\\\\\\\"arrival_time\\\\\\\":\\\\\\\"distinct count\\\\\\\",\\\\\\\"departure_time\\\\\\\":\\\\\\\"distinct count\\\\\\\",\\\\\\\"updated_at\\\\\\\":\\\\\\\"distinct count\\\\\\\",\\\\\\\"bearing\\\\\\\":\\\\\\\"avg\\\\\\\",\\\\\\\"current_stop_sequence\\\\\\\":\\\\\\\"sum\\\\\\\",\\\\\\\"direction_id_x\\\\\\\":\\\\\\\"sum\\\\\\\",\\\\\\\"direction_id_y\\\\\\\":\\\\\\\"sum\\\\\\\",\\\\\\\"route_type\\\\\\\":\\\\\\\"sum\\\\\\\",\\\\\\\"stop_sequence\\\\\\\":\\\\\\\"sum\\\\\\\",\\\\\\\"trip_direction_id_x\\\\\\\":\\\\\\\"sum\\\\\\\",\\\\\\\"trip_direction_id_y\\\\\\\":\\\\\\\"sum\\\\\\\",\\\\\\\"latitude\\\\\\\":\\\\\\\"sum\\\\\\\",\\\\\\\"longitude\\\\\\\":\\\\\\\"sum\\\\\\\",\\\\\\\"route_sort_order\\\\\\\":\\\\\\\"sum\\\\\\\",\\\\\\\"trip_bikes_allowed_x\\\\\\\":\\\\\\\"sum\\\\\\\",\\\\\\\"trip_bikes_allowed_y\\\\\\\":\\\\\\\"sum\\\\\\\",\\\\\\\"trip_wheelchair_accessible_x\\\\\\\":\\\\\\\"sum\\\\\\\",\\\\\\\"trip_wheelchair_accessible_y\\\\\\\":\\\\\\\"sum\\\\\\\",\\\\\\\"Direction\\\\\\\":\\\\\\\"count\\\\\\\"}\\\",\\\"render_time\\\":\\\"73.06500012427568\\\",\\\"settings\\\":\\\"true\\\"}\"}}},\"text/plain\":\"VisualEditorPart\"},\"4482072c-7d80-4e38-9d37-52c525ee70fa\":{\"application/vnd.maven.part+json\":{\"name\":\"CheckboxPart\",\"id\":\"4482072c-7d80-4e38-9d37-52c525ee70fa\",\"options\":{\"Checked\":{\"type\":\"Global\",\"expr\":\"AutoUpdate\",\"globals\":[\"AutoUpdate\"]},\"Label\":{\"typeName\":\"String\",\"value\":\"Auto-update?\"}}},\"text/plain\":\"VisualEditorPart\"},\"b85c483d-96a8-4295-9370-ae1d34cf72f7\":{\"application/vnd.maven.part+json\":{\"name\":\"AnimationSliderPart\",\"id\":\"b85c483d-96a8-4295-9370-ae1d34cf72f7\",\"options\":{\"Value\":{\"type\":\"Global\",\"expr\":\"rand\",\"globals\":[\"rand\"]},\"Timestep (ms)\":{\"typeName\":\"Number\",\"value\":12000},\"Loop\":{\"typeName\":\"Boolean\",\"value\":true}}},\"text/plain\":\"VisualEditorPart\"},\"f9d93dd1-8e8d-452f-92dd-1283b9c17764\":{\"application/vnd.maven.part+json\":{\"name\":\"PivotPart\",\"id\":\"f9d93dd1-8e8d-452f-92dd-1283b9c17764\",\"options\":{\"Config\":{\"typeName\":\"String\",\"value\":\"{\\\"class\\\":\\\"p-Widget\\\",\\\"view\\\":\\\"hypergrid\\\",\\\"row-pivots\\\":\\\"[]\\\",\\\"column-pivots\\\":\\\"[]\\\",\\\"filters\\\":\\\"[]\\\",\\\"sort\\\":\\\"[]\\\",\\\"style\\\":\\\"position: absolute; z-index: 0; top: 0px; left: 0px; width: 100%; height: 100%;\\\",\\\"updating\\\":\\\"true\\\",\\\"columns\\\":\\\"[\\\\\\\"null\\\\\\\"]\\\",\\\"aggregates\\\":\\\"{\\\\\\\"null\\\\\\\":\\\\\\\"count\\\\\\\"}\\\"}\"}}},\"text/plain\":\"VisualEditorPart\"},\"fc4bec04-9659-455c-b7f2-c5b217b22ed4\":{\"application/vnd.maven.part+json\":{\"name\":\"MavenTitlePart\",\"id\":\"fc4bec04-9659-455c-b7f2-c5b217b22ed4\",\"options\":{\"Name\":{\"typeName\":\"String\",\"value\":\"MBTA Live System Dashboard\"}}},\"text/plain\":\"VisualEditorPart\"}},\"metadata\":{},\"globals\":[{\"name\":\"AutoUpdate\",\"type\":\"Boolean\",\"value\":null},{\"name\":\"rand\",\"type\":\"Number\",\"value\":5}],\"localParts\":{},\"visual\":true}}"), raw=True)
del _json